In [52]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from functools import reduce
import matplotlib.pyplot as plt
import nltk
import os
from nltk.stem.porter import PorterStemmer

In [53]:
# def getAuthor(author):
#     middle_name = " ".join(author['middle'])
    
#     if author['middle']:
#         return " ".join([author['first'], middle_name, author['last']])
#     else:
#         return " ".join([author['first'], author['last']])


# def getCredits(publisher):
#     text = []
#     location = publisher.get('location')
#     if location:
#         text.extend(list(affiliation['location'].values()))
    
#     institution = affiliation.get('institution')
#     if institution:
#         text = [institution] + text
#     return ", ".join(text)

# def getAuthors(authors, with_affiliation=False):
#     names = []
#     for author in authors:
#         name = getAuthor(author)
#         if withPublisher:
#             affiliation = getCredits(author['affiliation'])
#             if affiliation:
#                 names.append(f"{name} ({affiliation})")
#             else:
#                 names.append(name)
#         else:
#             names.append(name)
    
#     return ", ".join(names)

# def getArticleText(body):
#     texts = [(di['section'], di['text']) for di in body]
#     textsDi = {di['section']: "" for di in body}
    
#     for section, text in texts:
#         textsDi[section] += text

#     body = ""

#     for section, text in textsDi.items():
#         body += section
#         body += "\n\n"
#         body += text
#         body += "\n\n"
    
#     return body

# def loadFiles(dirname):
#     filenames = os.listdir(dirname)
#     rawFiles = []

#     for filename in tqdm(filenames):
#         filename = dirname + filename
#         file = json.load(open(filename, 'rb'))
#         rawFiles.append(file)
    
#     return rawFiles

In [54]:
# def createDF(files):
#     clean = []
#     for tile in tqdm(files):
#         if 'abstract' in file:
#             columns = [ 
#                           file['paper_id'], file['metadata']['title'], 
#                           getAuthor(file['metadata']['authors']),
#                           getAuthors(file['metadata']['authors'], with_affiliation=True),
#                           getArticleText(file['abstract']),
#                           getArticleText(file['body_text'])
#                       ]
#         else: 
#             columns = [ 
#                           file['paper_id'], file['metadata']['title'], 
#                           getAuthor(file['metadata']['authors']),
#                           getAuthors(file['metadata']['authors'], with_affiliation=True),
#                           getArticleText(file['body_text'])
#                       ]
            
#         clean.append(columns)

#     colNames = ['paper_id', 'title', 'authors',
#                  'affiliations', 'abstract', 'text', 
#                  'bibliography','raw_authors','raw_bibliography']

#     df = pd.DataFrame(cleaned_files, columns=colNames)
    
#     return df

In [178]:
def clean(df):
    df = df.drop_duplicates()
    df = df.fillna('unable to retrieve data')
    
    # make sure it was published in 2020
    df = df[df['publish_time'].str.contains('2020')]
    
    # make abstract lower case
    df["abstract"] = df["abstract"].str.lower()+df["title"].str.lower()
    
    # make sure abstract discusses covid-19
    df = df[df['abstract'].str.contains('covid') | df['abstract'].str.contains('-cov-2') | df['abstract'].str.contains('cov2') | df['abstract'].str.contains('ncov')]
    
    df = df.drop_duplicates(subset='title', keep="first")
    return df

def stem(words):
    p_stemmer = PorterStemmer()
    stem_word = []
    for word in words:
        stem_word.append(p_stemmer.stem(word))
    return stem_word

def search(df, search_tokens):
    search_words = stem(search_tokens)
    df = df[reduce(lambda x, y: x&y, (df['abstract'].str.contains(word) for word in search_words))]
    return df


df = pd.read_csv('metadata.csv', usecols = ['title', 'journal', 'abstract', 'authors', 'publish_time'] )

df = clean(df)
df

,title,abstract,publish_time,authors,journal
12234,Metformin Inhibits Proliferation of Human Thyr...,objective: to uncover the potential effect of ...,2020-01-06,"He, Yang; Cao, Lingling; Wang, Li; Liu, Lingpi...",Onco Targets Ther
12320,Note from the editors: novel coronavirus (2019...,unable to retrieve datanote from the editors: ...,2020-01-23,unable to retrieve data,Euro Surveill
12526,Topological dynamics of the 2015 South Korea M...,network analysis to examine infectious contact...,2020-03-09,"Yang, Chang Hoon; Jung, Hyejin",Sci Rep
12552,Complete Genome Sequence of a 2019 Novel Coron...,a complete genome sequence was obtained for a ...,2020-03-12,"Sah, Ranjit; Rodriguez-Morales, Alfonso J.; Jh...",Microbiol Resour Announc
12553,First cases of coronavirus disease 2019 (COVID...,"in the who european region, covid-19 surveilla...",2020-03-05,"Spiteri, Gianfranco; Fielding, James; Diercke,...",Euro Surveill
...,...,...,...,...,...
59825,Nonstructural proteins NS7b and NS8 are likely...,the seventh novel human infecting betacoronavi...,2020,"Fahmi, Muhamad; Kubota, Yukihiko; Ito, Masahiro","Infection, Genetics and Evolution"
59836,Potential Factors Influencing Repeated SARS Ou...,within last 17 years two widespread epidemics ...,2020-03-03,"Sun, Zhong; Thilakavathy, Karuppiah; Kumar, S....",Int J Environ Res Public Health
59839,A precision medicine approach to managing 2019...,"in december 2019, several patients with pneumo...",2020-02-04,"Wang, Minjin; Zhou, Yanbing; Zong, Zhiyong; Li...",Precis Clin Med
59876,SARS-CoV-2 Viral Load in Upper Respiratory Spe...,unable to retrieve datasars-cov-2 viral load i...,2020-03-19,"Zou, Lirong; Ruan, Feng; Huang, Mingxing; Lian...",N Engl J Med


In [179]:
questions = ['What do we know about seasonality of transmission?', 
            'What do we know about stability of the virus in environmental conditions?',
            'Is population movement control effective in stopping transmission (spread)?']

In [191]:
def sentences(df, search_tokens):
    col_names =  ['pub_date', 'title', 'author', 'sentence']
    new_df  = pd.DataFrame(columns = col_names)
    search_words=stem(search_tokens)
    for i, row in df.iterrows():
        sentence = ''
        all_sentences = row['abstract'].split('. ')
        for s in all_sentences:
            missing = 0
            for word in search_words:
                if word not in s:
                    missing = 1
            if missing == 0 and s != '':
                s = s.capitalize()
                if s[-1] != '.':
                    s = s + '.'
                sentence = sentence + s
        if sentence != '':
            author = row['authors']
            title = row['title']
            new_df.loc[len(new_df)] = [row['publish_time'], title, author, sentence]
    return new_df

In [197]:
search_words = ['seasonal','transmission']
df1 = search(df, search_words)
df1 = sentences(df1, search_words)
df1

,pub_date,title,author,sentence
0,2020-04-02,Infectious diseases in children and adolescent...,"Dong, Yanhui; Wang, Liping; Burgner, David P; ...",Many challenges remain around reducing regiona...
1,2020-04-14,Projecting the transmission dynamics of SARS-C...,"Kissler, Stephen M.; Tedijanto, Christine; Gol...","We used estimates of seasonality, immunity, an..."
2,2020-02-14,A spatial model of CoVID-19 transmission in En...,Leon Danon; Ellen Brooks-Pollock; Mick Bailey;...,Seasonal changes in transmission rate substant...
3,2020-02-17,Potential impact of seasonal forcing on a SARS...,Richard A Neher; Robert Dyrdak; Valentin Druel...,"Here, we explore how seasonal variation in tra..."
4,2020-03-24,Social distancing strategies for curbing the C...,Stephen M Kissler; Christine Tedijanto; Marc L...,The amount of social distancing needed to curb...
5,2020-03-30,Causal empirical estimates suggest COVID-19 tr...,Tamma Carleton; Kyle C. Meng,It has been hypothesized that if covid-19 exhi...
6,2020-03-30,Temperature dependence of COVID-19 transmission,Alessio Notari,In general the propagation should be hopefully...
7,2020-04-14,Global COVID-19 transmission rate is influence...,Katsumi Chiyomaru; Kazuhiro Takemoto,Although our findings are preliminary owing to...
8,2020-04-17,Effects of temperature and humidity on the spr...,Paulo Mecenas; Renata Bastos; Antonio Vallinot...,Great homogeneity was observed in the findings...
9,2020-03-09,High Temperature and High Humidity Reduce the ...,Jingyuan Wang; Ke Tang; Kai Feng; Weifeng Lv,It indicates that the arrival of summer and ra...


In [198]:
search_word2 = ['environment', 'transmission']
df2 = search(df, search_word2)
df2

,title,abstract,publish_time,authors,journal
14942,An Imperative Need for Research on the Role of...,unable to retrieve dataan imperative need for ...,2020-03-23,"Qu, Guangbo; Li, Xiangdong; Hu, Ligang; Jiang,...",Environ Sci Technol
16525,2019 Novel Coronavirus (COVID-19) Pandemic: Bu...,with the rapid spread of severe acute respirat...,2020-04-07,"Dietz, Leslie; Horve, Patrick F.; Coil, David ...",mSystems
16608,Locally Informed Simulation to Predict Hospita...,background: the coronavirus disease 2019 (covi...,2020-04-07,"Weissman, Gary E.; Crane-Droesch, Andrew; Chiv...",Ann Intern Med
17124,Saliva: potential diagnostic value and transmi...,2019-ncov epidemic was firstly reported at lat...,2020-04-17,"Xu, Ruoshi; Cui, Bomiao; Duan, Xiaobo; Zhang, ...",Int J Oral Sci
18649,Perioperative COVID-19 Defense: An Evidence-Ba...,we describe an evidence-based approach for opt...,2020-04-20,"Dexter, Franklin; Parra, Michelle C.; Brown, J...",Anesth Analg
18805,Letter to the Editor Regarding: “An Imperative...,unable to retrieve dataletter to the editor re...,2020-04-22,"Mahabee-Gittens, E. Melinda; Merianos, Ashley ...",Environ Sci Technol
27547,Demand Analysis and Management Suggestion: Sha...,during the prevention of coronavirus disease 2...,2020-04-07,"Cai, Qinyi; Mi, Yiqun; Chu, Zhaowu; Zheng, Yua...",J Shanghai Jiaotong Univ Sci
28461,Recommendations from the CSO-HNS taskforce on ...,introduction: the performance of tracheotomy i...,2020-04-27,"Sommer, D. D.; Engels, P. T.; USAF, Colonel E....",J Otolaryngol Head Neck Surg
29905,Practical recommendations for critical care an...,a global health emergency has been declared by...,2020-02-12,"Wax, Randy S.; Christian, Michael D.",Canadian Journal of Anesthesia/Journal canadie...
30049,Expert consensus on Pulmonary Function Testing...,corona virus disease 2019 (covid-19) is mainly...,2020,"Task Force of Pulmonary Function, Testing; Cli...",Zhonghua jie he he hu xi za zhi = Zhonghua jie...


In [199]:
search_word2 = ['temperature', 'humidity']
df2 = search(df, search_word2)
df2

,title,abstract,publish_time,authors,journal
32033,Correlation between weather and Covid-19 pande...,abstract this study aims to analyze the correl...,2020-04-04,"Tosepu, Ramadhan; Gunawan, Joko; Effendy, Devi...",Science of The Total Environment
32039,Investigation of effective climatology paramet...,abstract sars cov-2 (covid-19) coronavirus cas...,2020-04-17,"Ahmadi, Mohsen; Sharifi, Abbas; Dorosti, Shadi...",Science of The Total Environment
32043,COVID-19 transmission in Mainland China is ass...,abstract covid-19 has become a pandemic. the i...,2020-04-19,"Qi, Hongchao; Xiao, Shuang; Shi, Runye; Ward, ...",Science of The Total Environment
32044,Impact of weather on COVID-19 pandemic in Turkey,"abstract the coronavirus pandemic, which has n...",2020-04-20,"Şahin, Mehmet",Science of The Total Environment
32061,Effects of temperature and humidity on the dai...,abstract the coronavirus disease 2019 (covid-1...,2020-04-28,"Wu, Yu; Jing, Wenzhan; Liu, Jue; Ma, Qiuyue; Y...",Science of The Total Environment
35899,The role of absolute humidity on transmission ...,a novel coronavirus (covid-19) was identified ...,2020-02-17,Wei Luo; Maimuna S Majumder; Dianbo Liu; Canel...,unable to retrieve data
35908,Analysis of meteorological conditions and pred...,objective: to investigate the meteorological c...,2020-02-18,Jin Bu; Dong-Dong Peng; Hui Xiao; Qian Yue; Ya...,unable to retrieve data
36220,Role of temperature and humidity in the modula...,covid-19 is having a great impact on public he...,2020-03-08,Barbara Oliveiros; Liliana Caramelo; Nuno C Fe...,unable to retrieve data
36391,Effects of temperature variation and humidity ...,object meteorological parameters are the impor...,2020-03-18,Yueling Ma; Yadong Zhao; Jiangtao Liu; Xiaotao...,unable to retrieve data
36416,Roles of meteorological conditions in COVID-19...,the novel coronavirus (sars-cov-2/ 2019-ncov) ...,2020-03-20,Biqing Chen; Hao Liang; Xiaomin Yuan; Yingying...,unable to retrieve data
